In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics


##### Reading data

In [2]:
dirpath = '/Users/parulgaba/Desktop/Capstone-Ethos/ConfidentialData/csvdata/'

data_path = '/Users/parulgaba/Desktop/Capstone-Ethos/ethos-retail-model/data/'

filename = data_path + 'regression_data/' + 'aggregated_summary_store_type_12_weeks.csv'
chunksize = 10 ** 5
rows=0
summary_df = pd.DataFrame()
for chunk in pd.read_csv(filename, chunksize=chunksize):
    summary_df=pd.concat([summary_df,chunk])
    rows+=chunk.shape[0]
    
summary_df.fillna(0)
print(summary_df.shape)
print (rows)

(118070, 47)
118070


In [3]:
summary_df.columns

Index(['period', 'item_no', 'state', 'store_type', 'brand', 'store_location',
       'city_type', 'region', 'quantity', 'purchase_quantity',
       'transfer_quantity', 'available_quantity', 'sales_quantity',
       'purchase_cost_amount', 'purchase_mrp', 'purchase_date',
       'stock_prevailing_mrp', 'store_in', 'product_group_code',
       'transfer_cost_amount', 'sales_department', 'days_to_sell',
       'num_of_customers', 'total_price', 'line_discount', 'crm_line_discount',
       'discount', 'tax', 'cost', 'billing', 'contribution', 'trade_incentive',
       'trade_incentive_value', 'total_contribution', 'case_size',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type', 'area_code'],
      dtype='object')

In [4]:
summary_df['stock_prevailing_mrp'] = summary_df['stock_prevailing_mrp'].div(1000000)
summary_df['billing'] = summary_df['billing'].div(1000000)


In [5]:
items_no_sales = summary_df.groupby(['item_no']).agg({'sales_quantity':'sum'}).reset_index()
unique_item_no_sales = items_no_sales[items_no_sales['sales_quantity'] == 0]['item_no'].unique()
summary_df = summary_df[~summary_df['item_no'].isin(unique_item_no_sales)]
print("Unique items removed with no sales at all for all 3 three years : " + str(len(unique_item_no_sales)))

Unique items removed with no sales at all for all 3 three years : 0


## Adding log S/So - Location code

In [6]:
#reading market shares
market_share=pd.read_excel(data_path + "market_share_encoded.xlsx", header=0,index_col=0)

#computing market size for each state-period
market_sizes=summary_df.groupby(['state','period']).agg({'sales_quantity':'sum'})
market_sizes=market_sizes.reset_index()
market_sizes=pd.merge(market_sizes,market_share, left_on='state', right_on='SubCode', how='left')#.drop('Attribute_x', axis=1)
market_sizes['Market Size']=market_sizes['sales_quantity'].div(market_sizes['Market Share'], axis=0)

#computing number of stores per state
x=summary_df.groupby(['state','period'])['store_type'].unique()
l=[]
store_nos=pd.DataFrame()
for i in range(len(x)):
    l.append([x.index[i][0],x.index[i][1],len(x[i])])
cols=['state','period','Store numbers']
store_nos = pd.DataFrame(l, columns=cols)

#merging market sizes with number of stores per market
market_sizes=pd.merge(market_sizes,store_nos, how='inner')

#computing market size per store
market_sizes['per store market']=market_sizes['Market Size']/market_sizes['Store numbers']

#adding market share per store-period to the main data
market_sizes=market_sizes[['state','period','per store market']]#extracting only relevant columns from market_sizes
merge_cols=['state','period']
summary_with_market_shares=pd.merge(summary_df,market_sizes, on=merge_cols,how='inner')

#computing So

# summary_with_market_shares['so'] = summary_with_market_shares['per store type market']-summary_with_market_shares['sales_quantity']

summary_with_market_shares['so']=summary_with_market_shares['per store market']-summary_with_market_shares['sales_quantity']
summary_with_market_shares = summary_with_market_shares[summary_with_market_shares['so'] != 0]

#computing log(S/So) [replacing zeros with 1e-08 so that logs dont create a problem]
summary_with_market_shares['so']=summary_with_market_shares['sales_quantity'].replace(0,10**(-5)).div(summary_with_market_shares['so'],axis=0)
summary_with_market_shares['so']=np.log(summary_with_market_shares['so'])



In [7]:
summary_with_market_shares = summary_with_market_shares[~(summary_with_market_shares['so'].isin([np.inf, -np.inf, np.nan]))]

In [8]:
#checking for NaNs
d=summary_with_market_shares[['sales_quantity','per store market','so']]
d[d.isna().any(axis=1)]

,sales_quantity,per store market,so


In [9]:
summary_with_market_shares['so'].head()

0   -8.042860
1   -8.042860
2   -6.095020
3   -6.943605
4   -8.042860
Name: so, dtype: float64

In [10]:
summary_with_market_shares['per store market'].head()

0    3112.5
1    3112.5
2    3112.5
3    3112.5
4    3112.5
Name: per store market, dtype: float64

In [11]:
#extracting specific columns

col=[ 'item_no','period', 'state', 'region',
       'brand', 'stock_prevailing_mrp', 'store_type', 'store_location', 'city_type',
       'available_quantity',  'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type','billing','sales_quantity','so']

summary_final=summary_with_market_shares.loc[:,col]
#df_north_final.fillna(0, inplace=True)

summary_final['item_no']=summary_final['item_no'].astype(str)
summary_final['period']=summary_final['period'].astype(str)
summary_final['case_shape']=summary_final['case_shape'].astype(str)


In [12]:
summary_final.shape

(118070, 24)

#### Defining a function for doing pareto analysis on features

The function combines all levels of a categorical features that cummulatively account for ~10% or less by Sales billings into a new level called "others". Features with less than 10 levels are not considered for pareto analysis.

In [13]:
def pareto(df,cols):
    lst=[]
    
    for col in cols:
                
        series=df.fillna(0).groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        mask=series.cumsum()/series.sum()>0.9 
        #nos=mask.value_counts()[1]
        mask=mask.iloc[:,0]
        levels=len(df[col].unique())
        
        if levels>10:
            df[col] = np.where(df[col].isin(series[mask].index),'Other',df[col])         
        new_levels=len(df[col].unique())

        freq=df[col].value_counts()/df[col].value_counts().sum()*100
        freq=freq.round(2)

        sale_qty=df.groupby([col]).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
        sale_qty=sale_qty/sale_qty.sum()*100
        sale_qty=sale_qty.round(2)
        try:
            Other_Sales_Qty=sale_qty['sales_quantity']['Other']
        except:
            Other_Sales_Qty=0
        
        bill=df.groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        bill=bill/bill.sum()*100
        bill=bill.round(2)
        try:
            Other_bill=bill['billing']['Other']
        except:
            Other_bill=0
        
        #comparison=mrp.merge(sale_qty, left_index=True, right_index=True)
        lst.append([col.upper(),levels, new_levels,Other_bill,Other_Sales_Qty])
        #print ("%s-Originally %d levels,combined %d levels into 'Other'.New Levels %d.By MRP,Other is %2.1f and by sale qty others is %2.1f"%(col.upper(),levels, levels-new_levels, new_levels,mrp['stock_prevailing_mrp']['Other'],sale_qty['sales_quantity']['Other']))
    
    cols=['Feature', 'Orig Levels', 'New Levels', 'Other%(Billing)', 'Other%(Sales Qty)']
    df1 = pd.DataFrame(lst, columns=cols)
    df1=df1.set_index("Feature")
    
    return df1,df

In [14]:
import numpy as np
cols=['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']
summary,summary_final_pareto=pareto(summary_final, cols)
summary


,Orig Levels,New Levels,Other%(Billing),Other%(Sales Qty)
Feature,,,,
CASE_SIZE_RANGE,13,7,10.79,15.43
GENDER,3,3,0.00,0.00
MOVEMENT,7,7,0.00,0.00
MATERIAL,58,8,10.15,14.24
DIAL_COLOR,49,10,11.01,9.27
STRAP_TYPE,67,9,10.66,16.39
STRAP_COLOR,48,7,13.71,24.53
PRECIOUS_STONE,8,8,0.00,0.00
GLASS,7,7,0.00,0.00


In [15]:
summary_final_pareto.shape

(118070, 24)

#### Creating dummy variables

In [16]:
#creating dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type']
summary_final_dummies=pd.get_dummies(data=summary_final_pareto, columns=cols)

print('Done')

Done


In [17]:
summary_final_dummies.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 118070 entries, 0 to 118069
Columns: 196 entries, item_no to watch_type_smart watch
dtypes: float64(3), int64(2), object(2), uint8(189)
memory usage: 28.5+ MB


In [18]:
#for So as the target variable
#creating seperate df for independent and dependent features
y=summary_final_dummies.loc[:, summary_final_dummies.columns == 'so']
X=summary_final_dummies.drop(columns =['sales_quantity','item_no','billing', 'so', 'period'])



In [19]:
#checking for duplicate column names
duplicate_columns = summary_final_dummies.columns[summary_final_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [20]:
#performing train and test split on data
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


"""
split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)
"""

'\nsplit=0.2\ntest=int(len(X)*split)\ntrain=len(X)-test\nX_train=X.head(train)\ny_train=y.head(train)\nX_test=X.tail(test)\ny_test=y.tail(test)\n'

In [21]:
#!pip install -U imbalanced-learn
"""
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=11915008)
X_bal, y_bal = ros.fit_resample(X_train, y_train)

print('Done')
"""

"\nfrom imblearn.over_sampling import RandomOverSampler\nros = RandomOverSampler(random_state=11915008)\nX_bal, y_bal = ros.fit_resample(X_train, y_train)\n\nprint('Done')\n"

### Regression Modelling

In [22]:
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()  
reg_model.fit(X_train, y_train)

LinearRegression()

In [23]:
preds = reg_model.predict(X_test)

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
rmse = np.sqrt(mean_squared_error(y_test, preds))


In [25]:
r2=r2_score(y_test, preds)
print("Linear regression RMSE: %.2f, Test R2: %.2f" % (rmse,r2))

Linear regression RMSE: 0.41, Test R2: 0.74


## XGBoost

In [26]:
#!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

/Users/parulgaba/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/parulgaba/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [27]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))


In [28]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,preds)
print("Xgboost RMSE: %.2f, Test R2: %.2f" % (rmse,r2))

Xgboost RMSE: 3.08, Test R2: -13.46


In [29]:
params = {'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'subsample': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'colsample_bylevel':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'min_child_weight':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'n_estimator':[3,5,7,10],
          'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          'max_depth': [3,4,5,6,7,8,10,12,15], 
          'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
         }

In [30]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
#scoring='r2'
scoring='neg_mean_squared_error'
random_search=RandomizedSearchCV(xg_reg,param_distributions=params,scoring=scoring,n_iter=20,cv=5,verbose=3)

In [31]:
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[01:52:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9, score=-0.154, total=  59.8s
[CV] subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   59.8s remaining:    0.0s


[01:53:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9, score=-0.152, total= 1.0min
[CV] subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s


[01:54:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9, score=-0.152, total=  59.5s
[CV] subsample=0.4, reg_lambda=0.8, reg_alpha=0.3, n_estimator=10, min_child_weight=3, max_depth=15, learning_rate=0.2, colsample_bytree=0.7, colsample_bylevel=0.9 
[01:55:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but s

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 24.6min finished


[02:16:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                                             0.5, 0.6, 0.7, 0.8,
                                                             0.9],
                                        'l

In [32]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=10,
             min_child_weight=7, missing=nan, monotone_constraints='()',
             n_estimator=5, n_estimators=100, n_jobs=0, num_parallel_tree=1,
             random_state=0, reg_alpha=0.7, reg_lambda=0.7, scale_pos_weight=1,
             subsample=0.3, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [33]:
random_search.best_params_

{'subsample': 0.3,
 'reg_lambda': 0.7,
 'reg_alpha': 0.7,
 'n_estimator': 5,
 'min_child_weight': 7,
 'max_depth': 10,
 'learning_rate': 0.15,
 'colsample_bytree': 0.5,
 'colsample_bylevel': 0.5}

In [34]:
#best model based on the output of random_search.best_estimator_
best_gb=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=12,
             min_child_weight =7, missing=np.nan,
              n_estimator=3, n_estimators=100,
             n_jobs=0, num_parallel_tree=1, objective='reg:squarederror',
             random_state=0, reg_alpha=0.5, reg_lambda=0.3, scale_pos_weight=1,
             subsample=0.2, tree_method='exact', validate_parameters=1,
             verbosity=0)

In [35]:
from sklearn.model_selection import cross_val_score as cvs
score=cvs(best_gb,X_train,y_train,cv=2,scoring='r2')

In [36]:
score

array([0.7773132 , 0.77493445])

In [37]:
score.mean()

0.7761238239930711

In [38]:
best_gb.fit(X_train,y_train)
preds = best_gb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.380639


In [39]:
from sklearn.metrics import r2_score
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2=r2_score(y_test,preds)
print("RMSE: %.2f, R2: %.2f" % (rmse,r2))

RMSE: 0.38, R2: 0.78
